In [1]:
import os
import pandas as pd
import numpy as np
import gc
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
import mlcrate as mlc
import pickle as pkl
from tensorflow.keras.layers import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dot, Reshape, Add, Subtract
from keras import backend as K
from keras import regularizers 
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2
from sklearn.base import clone
from typing import Dict
import matplotlib.pyplot as plt
from scipy import stats
from tensorflow.keras.losses import Loss
from tensorflow.keras import backend as K
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import TimeSeriesSplit, StratifiedKFold, KFold, GroupKFold
from tqdm import tqdm
from tensorflow.python.ops import math_ops

In [2]:
%%time
n_features = 300
features = [f'f_{i}' for i in range(n_features)]
feature_columns = ['investment_id', 'time_id'] + features
train = pd.read_pickle('../input/ubiquant-market-prediction-half-precision-pickle/train.pkl')
train.head()

CPU times: user 613 ms, sys: 2.07 s, total: 2.68 s
Wall time: 19 s


,investment_id,time_id,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,...,f_291,f_292,f_293,f_294,f_295,f_296,f_297,f_298,f_299,target
0,1,0,0.932617,0.113708,-0.402100,0.378418,-0.203979,-0.413574,0.965820,1.230469,...,-1.095703,0.200073,0.819336,0.941406,-0.086792,-1.086914,-1.044922,-0.287598,0.321533,-0.300781
1,2,0,0.811035,-0.514160,0.742188,-0.616699,-0.194214,1.771484,1.427734,1.133789,...,0.912598,-0.734375,0.819336,0.941406,-0.387695,-1.086914,-0.929688,-0.974121,-0.343506,-0.231079
2,6,0,0.394043,0.615723,0.567871,-0.607910,0.068909,-1.083008,0.979492,-1.125977,...,0.912598,-0.551758,-1.220703,-1.060547,-0.219116,-1.086914,-0.612305,-0.113953,0.243652,0.568848
3,7,0,-2.343750,-0.011871,1.875000,-0.606445,-0.586914,-0.815918,0.778320,0.299072,...,0.912598,-0.266357,-1.220703,0.941406,-0.608887,0.104919,-0.783203,1.151367,-0.773438,-1.064453
4,8,0,0.842285,-0.262939,2.330078,-0.583496,-0.618164,-0.742676,-0.946777,1.230469,...,0.912598,-0.741211,-1.220703,0.941406,-0.588379,0.104919,0.753418,1.345703,-0.737793,-0.531738


In [3]:
investment_id = train.pop("investment_id")
investment_id.head()

0    1
1    2
2    6
3    7
4    8
Name: investment_id, dtype: uint16

In [4]:
_ = train.pop("time_id")
y = train.pop("target")
y.head()

0   -0.300781
1   -0.231079
2    0.568848
3   -1.064453
4   -0.531738
Name: target, dtype: float16

In [5]:
%%time
investment_ids = list(investment_id.unique())
investment_id_size = len(investment_ids) + 1
investment_id_lookup_layer = layers.IntegerLookup(max_tokens=investment_id_size)
with tf.device("cpu"):
    investment_id_lookup_layer.adapt(investment_id)

2022-04-18 17:03:54.989118: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-04-18 17:03:55.110559: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


CPU times: user 1min 1s, sys: 9.7 s, total: 1min 11s
Wall time: 44.7 s


In [6]:
def preprocess(X, y):
    print(X)
    print(y)
    return X, y
def make_dataset(feature, investment_id, y, batch_size=1024, mode="train"):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature), y))
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(256)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [7]:
def get_model():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model


def get_model2():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)    
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
   # investment_id_x = layers.Dropout(0.65)(investment_id_x)
   
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dense(256, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.65)(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
   # x = layers.Dropout(0.2)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
  #  x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.75)(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model


def get_model3():
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float32)
    
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    investment_id_x = layers.Dense(32, activation='swish')(investment_id_x)
    investment_id_x = layers.Dropout(0.5)(investment_id_x)
    #investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(128, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.5)(feature_x)
    feature_x = layers.Dense(64, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(64, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.5)(x)
    output = layers.Dense(1)(x)
    output = tf.keras.layers.BatchNormalization(axis=1)(output)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model

def get_model_5():
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    ## feature ##
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.1)(feature_x)
    ## convolution 1 ##
    feature_x = layers.Reshape((-1,1))(feature_x)
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 2 ##
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 3 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 4 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 5 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=2, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## flatten ##
    feature_x = layers.Flatten()(feature_x)
    
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(feature_x)
    
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model

def get_model_6():
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    features_x = layers.GaussianNoise(0.1)(features_inputs)
    ## feature ##
    feature_x = layers.Dense(256, activation='swish')(features_x)
    feature_x = layers.Dropout(0.1)(feature_x)
    ## convolution 1 ##
    feature_x = layers.Reshape((-1,1))(feature_x)
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 2 ##
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 3 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 4 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 5 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=2, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## flatten ##
    feature_x = layers.Flatten()(feature_x)
 
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(feature_x)
    
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model

def get_model_7():
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    
    ## Dense 1 ##
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.1)(feature_x)
    ## convolution 1 ##
    feature_x = layers.Reshape((-1,1))(feature_x)
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 2 ##
    feature_x = layers.Conv1D(filters=16, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution 3 ##
    feature_x = layers.Conv1D(filters=64, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)

    ## convolution2D 1 ##
    feature_x = layers.Reshape((64,64,1))(feature_x)
    feature_x = layers.Conv2D(filters=32, kernel_size=4, strides=1, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution2D 2 ##
    feature_x = layers.Conv2D(filters=32, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)
    ## convolution2D 3 ##
    feature_x = layers.Conv2D(filters=32, kernel_size=4, strides=4, padding='same')(feature_x)
    feature_x = layers.BatchNormalization()(feature_x)
    feature_x = layers.LeakyReLU()(feature_x)

    ## flatten ##
    feature_x = layers.Flatten()(feature_x)
    ## Dense 3 ##
    x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(feature_x)
    ## Dense 4 ##
    x = layers.Dropout(0.1)(x)
    ## Dense 5 ##    
    x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    ## Dense 6 ##
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.1)(x)
    ## Dense 7 ##
    output = layers.Dense(1)(x)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001), loss='mse', metrics=['mse', "mae", "mape", rmse])
    return model

gc.collect()

754

In [8]:
models = []

for i in range(5):
    model = get_model()
    model.load_weights(f'../input/dnn-base/model_{i}')
    models.append(model)

for i in range(10):
    model = get_model2()
    model.load_weights(f'../input/train-dnn-v2-10fold/model_{i}')
    models.append(model)
    
    
for i in range(10):
    model = get_model3()
    model.load_weights(f'../input/dnnmodelnew/model_{i}')
    models.append(model)
    
    
models2 = []
    
for i in range(5):
    model = get_model_5()
    model.load_weights(f'../input/prediction-including-spatial-info-with-conv1d/model_{i}.tf')
    models2.append(model)

for i in range(5):
    model = get_model_6()
    model.load_weights(f'../input/ump-conv1d-with-gaussian-noise-train/model_{i}.tf')
    models2.append(model)

for i in range(5):
    model = get_model_7()
    model.load_weights(f'../input/../input/ump-prediction-with-conv2ddata/model_{i}.tf')
    models2.append(model)

2022-04-18 17:04:40.272785: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open ../input/dnn-base/model_0: Data loss: file is too short to be an sstable: perhaps your file is in a different file format and you need to use a different restore operator?
2022-04-18 17:04:40.642192: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open ../input/dnn-base/model_1: Data loss: file is too short to be an sstable: perhaps your file is in a different file format and you need to use a different restore operator?
2022-04-18 17:04:40.979757: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open ../input/dnn-base/model_2: Data loss: file is too short to be an sstable: perhaps your file is in a different file format and you need to use a different restore operator?
2022-04-18 17:04:41.332328: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open ../input/dnn-base/model_3: Data loss: file is too short to be an sstable: perhaps your file is in a different fi

In [9]:
def get_model_dr04():
    features_inputs = tf.keras.Input((300, ), dtype=tf.float32)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.4)(feature_x)
    feature_x = layers.Dense(128, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.4)(feature_x)
    feature_x = layers.Dense(64, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([feature_x])
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(64, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(1)(x)
    output = tf.keras.layers.BatchNormalization(axis=1)(output)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001),  loss = correlationLoss, metrics=[correlationMetric])
    return model

dr=0.3

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    print("Name:", gpu.name, "  Type:", gpu.device_type)

n_features = 300
features = [f'f_{i}' for i in range(n_features)]

def preprocess(X, y):
    return X, y
def make_dataset(feature, y, batch_size=1024, mode="train"):
    ds = tf.data.Dataset.from_tensor_slices((feature, y))
    ds = ds.map(preprocess)
    if mode == "train":
        ds = ds.shuffle(512)
#     ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def correlationMetric(x, y, axis=-2):
    """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    return tf.constant(1.0, dtype=x.dtype) - corr


def correlationLoss(x,y, axis=-2):
    """Loss function that maximizes the pearson correlation coefficient between the predicted values and the labels,
    while trying to have the same mean and variance"""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xsqsum = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    ysqsum = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xsqsum * ysqsum)
    return tf.convert_to_tensor( K.mean(tf.constant(1.0, dtype=x.dtype) - corr ) , dtype=tf.float32 )

def correlationMetric_01mse(x, y, axis=-2):
    """Metric returning the Pearson correlation coefficient of two tensors over some axis, default -2."""
    x = tf.convert_to_tensor(x)
    y = math_ops.cast(y, x.dtype)
    n = tf.cast(tf.shape(x)[axis], x.dtype)
    xsum = tf.reduce_sum(x, axis=axis)
    ysum = tf.reduce_sum(y, axis=axis)
    xmean = xsum / n
    ymean = ysum / n
    xvar = tf.reduce_sum( tf.math.squared_difference(x, xmean), axis=axis)
    yvar = tf.reduce_sum( tf.math.squared_difference(y, ymean), axis=axis)
    cov = tf.reduce_sum( (x - xmean) * (y - ymean), axis=axis)
    corr = cov / tf.sqrt(xvar * yvar)
    return tf.constant(1.0, dtype=x.dtype) - corr

gc.collect()

# list(GroupKFold(5).split(train , groups = train.index))[0]
def pearson_coef(data):
    return data.corr()['target']['preds']

def evaluate_metric(valid_df):
    return np.mean(valid_df[['time_id_', 'target', 'preds']].groupby('time_id').apply(pearson_coef))

 
models3 = []

for index in range(10):
    model = get_model_dr04()
    model.load_weights(f"../input/ubiquantmodels3/model_kf-20220412T021941Z-001/model_kf/model_dr4_corr_10_{index}.tf")
    models3.append(model)
    
    


In [10]:
investment_id[~investment_id.isin([85, 905, 2558, 3662, 2800, 1415])]

0             1
1             2
2             6
3             7
4             8
           ... 
3141405    3768
3141406    3769
3141407    3770
3141408    3772
3141409    3773
Name: investment_id, Length: 3141374, dtype: uint16

In [11]:
investment_id2 = investment_id[~investment_id.isin([85, 905, 2558, 3662, 2800, 1415])]

investment_ids2 = list(investment_id2.unique())
investment_id_size2 = len(investment_ids2) + 1
investment_id_lookup_layer2 = layers.IntegerLookup(max_tokens=investment_id_size2)
investment_id_lookup_layer2.adapt(pd.DataFrame({"investment_ids":investment_ids}))

In [12]:
class get_models():
  def __init__(self,model_name,loss):
    self.model_name = model_name
    if loss == 'mse':
      self.loss = loss
    elif loss == 'corr':
      self.loss = correlationLoss

  def get_train_model(self):
    if self.model_name == 'dnn_model1':
      return self.get_dnn_model1()

    if self.model_name == 'lstm_model1':
      return self.get_lstm_model1()
    
    if self.model_name == 'model_dr4':
      return self.get_model_dr04()
    
    if self.model_name == 'dnn_model2':
      return self.get_dnn_model2()

    if self.model_name == 'dnn_model3':
      return self.get_dnn_model3()
  
  def get_dnn_model1(self):
      investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
      features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
      
      investment_id_x = investment_id_lookup_layer(investment_id_inputs)
      investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
      investment_id_x = layers.Reshape((-1, ))(investment_id_x)
      investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
      investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
      investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
      
      feature_x = layers.Dense(256, activation='swish')(features_inputs)
      feature_x = layers.Dense(256, activation='swish')(feature_x)
      feature_x = layers.Dense(256, activation='swish')(feature_x)
      
      x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
      x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
      x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
      x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
      output = layers.Dense(1)(x)
      # output = tf.keras.layers.BatchNormalization(axis=1)(output)
      rmse = keras.metrics.RootMeanSquaredError(name="rmse")
      model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
      model.compile(optimizer=tf.optimizers.Adam(0.001), loss=self.loss, metrics=['mse'])
      return model


  def get_dnn_model2(self):
      investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
      features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
      
      investment_id_x = investment_id_lookup_layer(investment_id_inputs)
      investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
      investment_id_x = layers.Reshape((-1, ))(investment_id_x)
      investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)    
      investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
      investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
      investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
    # investment_id_x = layers.Dropout(0.65)(investment_id_x)
    
      
      feature_x = layers.Dense(256, activation='swish')(features_inputs)
      feature_x = layers.Dense(256, activation='swish')(feature_x)
      feature_x = layers.Dense(256, activation='swish')(feature_x)
      feature_x = layers.Dense(256, activation='swish')(feature_x)
      feature_x = layers.Dropout(0.65)(feature_x)
      
      x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
      x = layers.Dense(512, activation='swish', kernel_regularizer="l2")(x)
    # x = layers.Dropout(0.2)(x)
      x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
    #  x = layers.Dropout(0.4)(x)
      x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
      x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
      x = layers.Dropout(0.75)(x)
      output = layers.Dense(1)(x)
      rmse = keras.metrics.RootMeanSquaredError(name="rmse")
      model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
      model.compile(optimizer=tf.optimizers.Adam(0.001), loss=self.loss, metrics=['mse'])
      return model


  def get_dnn_model3(self):
      investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
      features_inputs = tf.keras.Input((300, ), dtype=tf.float32)
      
      investment_id_x = investment_id_lookup_layer(investment_id_inputs)
      investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
      investment_id_x = layers.Reshape((-1, ))(investment_id_x)
      investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
      investment_id_x = layers.Dropout(0.5)(investment_id_x)
      investment_id_x = layers.Dense(32, activation='swish')(investment_id_x)
      investment_id_x = layers.Dropout(0.5)(investment_id_x)
      #investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
      
      feature_x = layers.Dense(256, activation='swish')(features_inputs)
      feature_x = layers.Dropout(0.5)(feature_x)
      feature_x = layers.Dense(128, activation='swish')(feature_x)
      feature_x = layers.Dropout(0.5)(feature_x)
      feature_x = layers.Dense(64, activation='swish')(feature_x)
      
      x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
      x = layers.Dropout(0.5)(x)
      x = layers.Dense(64, activation='swish', kernel_regularizer="l2")(x)
      x = layers.Dropout(0.5)(x)
      x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
      x = layers.Dropout(0.5)(x)
      x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)
      x = layers.Dropout(0.5)(x)
      output = layers.Dense(1)(x)
      output = tf.keras.layers.BatchNormalization(axis=1)(output)
      rmse = keras.metrics.RootMeanSquaredError(name="rmse")
      model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
      model.compile(optimizer=tf.optimizers.Adam(0.001), loss=self.loss, metrics=['mse'])
      return model
      
  def get_lstm_model1(self):
      investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
      features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
      
      investment_id_x = investment_id_lookup_layer(investment_id_inputs)
      investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
      investment_id_x = layers.Reshape((-1, ))(investment_id_x)
      investment_id_x = layers.Dense(128, activation='swish')(investment_id_x)
      investment_id_x = layers.Dense(64, activation='swish')(investment_id_x)
      investment_id_x = layers.Dense(32, activation='swish')(investment_id_x)
      
      feature_x = layers.Dense(256, activation='swish')(features_inputs)
      feature_x = layers.Dense(128, activation='swish')(feature_x)
      feature_x = layers.Dense(64, activation='swish')(feature_x)
      
      x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
      x = layers.Dense(256, activation='swish', kernel_regularizer="l2")(x)
      # x = layers.Dense(256, activation='swish', kernel_regularizer="l2")(x)
      x = layers.Dense(128, activation='swish', kernel_regularizer="l2")(x)
      x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
      # x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)


      x = layers.BatchNormalization(name='batch_norm1')(x)
      x = layers.Dense(256, activation='swish', name='dense1')(x)
      x = layers.Dropout(0.1, name='dropout1')(x)
      x = layers.Reshape((1, -1), name='reshape1')(x)
      x = layers.BatchNormalization(name='batch_norm2')(x)
      x = layers.LSTM(128, return_sequences=True, activation='relu', name='lstm1')(x)
      x = layers.LSTM(16, return_sequences=False, activation='relu', name='lstm2')(x)


      output = layers.Dense(1)(x)
      rmse = keras.metrics.RootMeanSquaredError(name="rmse")
      model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
      model.compile(optimizer=tf.optimizers.Adam(0.001), loss=self.loss, metrics=['mse'])
      return model

  def get_model_dr04(self):
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    features_inputs = tf.keras.Input((300, ), dtype=tf.float32)
    
    feature_x = layers.Dense(256, activation='swish')(features_inputs)
    feature_x = layers.Dropout(0.4)(feature_x)
    feature_x = layers.Dense(128, activation='swish')(feature_x)
    feature_x = layers.Dropout(0.4)(feature_x)
    feature_x = layers.Dense(64, activation='swish')(feature_x)
    
    x = layers.Concatenate(axis=1)([feature_x])
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(64, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(32, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(16, activation='swish', kernel_regularizer="l2")(x)
    x = layers.Dropout(0.4)(x)
    output = layers.Dense(1)(x)
    output = tf.keras.layers.BatchNormalization(axis=1)(output)
    rmse = keras.metrics.RootMeanSquaredError(name="rmse")
    model = tf.keras.Model(inputs=[investment_id_inputs,features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.001),  loss =self.loss, metrics=[correlationMetric])
    return model

def get_submit_models(models,model_name,loss,split_num=5):
    for index in range(split_num):
        model = get_models(model_name=model_name,loss=loss).get_train_model()
        model.load_weights(f'../input/ubiquantmodels3/model_kf-20220412T021941Z-001/model_kf/{model_name}_{loss}_{split_num}_{index}.tf')
        models.append(model)
    return models

In [13]:

models_final = []

models_final = get_submit_models(models_final,'dnn_model3','corr')
models_final = get_submit_models(models_final,'model_dr4','corr',10)
models_final = get_submit_models(models_final,'model_dr4','mse',10)
models_final = get_submit_models(models_final,'dnn_model1','mse')
models_final = get_submit_models(models_final,'dnn_model2','corr')
models_final = get_submit_models(models_final,'lstm_model1','corr')



In [14]:
import joblib
tp = '../input/ubiquantmodels3/stack_model-20220415T025221Z-001/stack_model/'
stack_models =  [joblib.load(tp + stack_model_path) for stack_model_path in os.listdir(tp)]
def inference_models4(models, ds , ifstack=False):
    y_preds = []
    stack_preds = []
    count = 0
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
        if count < 20:
            if len(y_preds) % 5 == 0:
                stack_preds.append(np.mean(y_preds[count:count+5],axis=0))
                count += 5
        else:
            if len(y_preds) % 10 == 0:
                stack_preds.append(np.mean(y_preds[count:count+10],axis=0))
                count += 10
    stack_preds_last = []
    stack_preds = np.hstack(stack_preds)
    for stack_model in stack_models:
        stack_preds_last.append(stack_model.predict(stack_preds))
        return np.mean(stack_preds_last,axis=0)


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:333: UserWarning: Trying to unpickle estimator Ridge from version 1.0.2 when using version 1.0.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [15]:
def get_model_corr(ft_units, x_units, x_dropout):
    
    # investment_id
    investment_id_inputs = tf.keras.Input((1, ), dtype=tf.uint16)
    investment_id_x = investment_id_lookup_layer(investment_id_inputs)
    investment_id_x = layers.Embedding(investment_id_size, 32, input_length=1)(investment_id_x)
    investment_id_x = layers.Reshape((-1, ))(investment_id_x)
    investment_id_x = layers.Dense(128, activation='swish')(investment_id_x)
    investment_id_x = layers.Dense(128, activation='swish')(investment_id_x) 
    investment_id_x = layers.Dense(128, activation='swish')(investment_id_x)
    
    # features_inputs
    features_inputs = tf.keras.Input((300, ), dtype=tf.float16)
    bn = tf.keras.layers.BatchNormalization()(features_inputs)
    gn = tf.keras.layers.GaussianNoise(0.035)(bn)
    feature_x = layers.Dense(300, activation='swish')(gn)
    feature_x = tf.keras.layers.Dropout(0.5)(feature_x)
    
    for hu in ft_units:
        feature_x = layers.Dense(hu, activation='swish')(feature_x)
#         feature_x = tf.keras.layers.Activation('swish')(feature_x)
        feature_x = tf.keras.layers.Dropout(0.35)(feature_x)
    
    x = layers.Concatenate(axis=1)([investment_id_x, feature_x])
    
    for i in range(len(x_units)):
        x = tf.keras.layers.Dense(x_units[i], kernel_regularizer="l2")(x) 
        x = tf.keras.layers.Activation('swish')(x)
        x = tf.keras.layers.Dropout(x_dropout[i])(x)
        
    output = layers.Dense(1)(x)
    model = tf.keras.Model(inputs=[investment_id_inputs, features_inputs], outputs=[output])
    model.compile(optimizer=tf.optimizers.Adam(0.0001), loss=correlationLoss, 
                  metrics=['mse', "mae", correlation])
    return model


params = {
#     'num_columns': len(features), 
    'ft_units': [150, 75, 150 ,200],
    'x_units': [512, 256, 128, 32],
    'x_dropout': [0.44, 0.4, 0.33, 0.2] #4, 3, 2, 1
#           'lr':1e-3, 
         }

In [16]:
del train
del investment_id
del y

In [17]:
def preprocess_test(investment_id, feature):
    return (investment_id, feature), 0

def preprocess_test_s(feature):
    return (feature), 0

def make_test_dataset(feature, investment_id, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def make_test_dataset2(feature, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((feature)))
    ds = ds.batch(batch_size).cache().prefetch(tf.data.AUTOTUNE)
    return ds

def inference(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append(y_pred)
    return np.mean(y_preds, axis=0)

from sklearn.decomposition import PCA
pca = PCA(n_components=1)


def make_test_dataset3(feature, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices((feature))
    ds = ds.map(preprocess_test_s)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def make_test_dataset4(feature, investment_id, batch_size=1024):
    ds = tf.data.Dataset.from_tensor_slices(((investment_id, feature)))
    ds = ds.map(preprocess_test)
    ds = ds.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
    return ds

def infer(models, ds):
    y_preds = []
    for model in models:
        y_pred = model.predict(ds)
        y_preds.append((y_pred-y_pred.mean())/y_pred.std())
    
    return np.mean(y_preds, axis=0)

In [18]:
models_final

In [19]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test() 
for (test_df, sample_prediction_df) in iter_test:
    ds = make_test_dataset(test_df[features], test_df["investment_id"])
    prediction1 = inference(models, ds)
    ds2 = make_test_dataset2(test_df[features])
    prediction2 = inference(models2, ds2)
    ds3 = make_test_dataset3(test_df[features])
    prediction3 = infer(models3, ds3)
    ds4 = make_test_dataset4(test_df[features], test_df["investment_id"])
    prediction4 = inference_models4(models_final, ds4,True)
    sample_prediction_df['target'] = 0.15 * prediction1 + 0.4 * prediction2 + 0.15 * prediction3 + 0.3 * prediction4
    env.predict(sample_prediction_df) 
    display(sample_prediction_df)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  "X does not have valid feature names, but"


,row_id,target
0,1220_1,-0.210231
1,1220_2,0.084718


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  "X does not have valid feature names, but"


,row_id,target
0,1221_0,-0.007128
1,1221_1,-0.008520
2,1221_2,-0.203045


/opt/conda/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  "X does not have valid feature names, but"


,row_id,target
0,1222_0,-0.133750
1,1222_1,0.046075
2,1222_2,-0.099781


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:45: RuntimeWarning: invalid value encountered in true_divide
/opt/conda/lib/python3.7/site-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  "X does not have valid feature names, but"


,row_id,target
0,1223_0,NaN
